# 강화학습 실습

- 다중 슬롯 머신
  - 가치 계산용 알고리즘
    - UCB1
    - ε-greedy
  - 강화학습 입문형

- 미로게임
  - 정책 계산 (정책반복법, 가치계산법)
    - 알고리즘
      - 정책경사법
      - Sarsa
      - Q-Learning
- 카트풀 게임
  - openAI-gym 제공 받아서 활용
    - 정책 계산(정책반복법, 가치계산법)
      - 알고리즘
        - DQN 활용(tensorflow + keras)

# 다중 슬롯머신

- 목적
  - 강화학습, 스타일을 익힌다
  - 강화학습의 절차 습득 및 요소 정의, 학습 스타일 습득

- 환경
  - 게임의 룰
  - 슬롯머신의 ARM(팔)이 여러개 존재할 수 있다(현실세계에서 슬롯머신은 통상 ARM이 한개이다)
  - 각각의 ARM은 보상치가 다르다. 각각의 확률이 정해져 있다
    - 1번 ARM 10% 확률로 보상제공 
    - 2번 ARM 50% 확률로 보상제공
    - 각각의 보상치는 동일하게 1이다
  - 에이전트는 게임을 시작하면 각 팔에 대해서는 보상 확률을 모른다. 게임이 세팅되면 확률은 정해진다
    - 보상의 크기는 중요하지 않다(1), 보상을 받는가 OR 못받는가에만 집중

- 에이전트의 목적
  - 어떤 ARM을 선택해야 최대 보상을 받는가? 이 ARM을 찾아내는 것이 관건 -> 이것이 학습의 목적
- 에이전트의 행동
  - 여러개의 ARM 중 한개를 선택(당긴다)
  - ARM의 확률에 따라 보상을 지급(0 OR 1)
  - 게임 한 판이 종료된다

|강화 학습 요소|다중 슬롯 머신 게임|
|--:|:--|
|에이전트|슬롯 머신을 내리는 사람|
|환경|다중 슬롯 머신|
|목적|많은 보상 획득, 제한된 횟수|
|행동|여러 개의 ARM중 한개를 선택(당긴다)|
|에피소드|ARM을 당기면, 게임 종료|
|상태|없음|
|보상|1.0 혹은 0.0|
|수익|보상의 총합(즉시보상 + 지연보상(0))|
|학습방법|UCB1, ε-greedy|
|파라미터갱신주기|1회 행동으로 게임 종료되면 즉시 진행|
|정책|다음 슬롯머신 ARM 선택시 판단용|

# 정책

- **탐색과 이용의 트레이드 오프 문제**
  - 정보수집을 위해서 탐색을 수행하면, 언젠가 모든 ARM에 대한 정보를 알게 된다. 만약 제한된 횟수에서 탐색의 수가 늘어나면, 가장 높은 팔만 집중적으로 당기는 수익보다는 적을 것이다

  - 반대 관점, 몇 번 선택 후, 가장 높은 보상을 받은 ARM을 집중적으로 선택한다면 => 나중에 더 높은 보상을 주는 ARM을 선택하지 못해서 더 적은 보상을 받을 수도 있다

# 구현

## 구조

```
- SlotMachineGame
  L SlotArm Class : 슬롯머신의 팔
  L GameEngine Class : 알고리즘 사용의 표준 인터페이스 제공
    L EpsilonGreedyEngine Class : 알고리즘 1
    L UCB1Engine Class : 알고리즘 2
  L GameSimulator Class or function : 게임 가동, 시뮬레이션 진행 
```

# 필요 모듈

In [ ]:
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# SlotArm Class

In [ ]:
m = [random.random() for n in range(1000)]
print(m)

[0.6910484504332359,
 0.05970303635433771,
 0.9845649200180232,
 0.4454145521091568,
 0.15779530536833064,
 0.7012502215361283,
 0.42130570426659053,
 0.5945451770856083,
 0.2736565066757721,
 0.35307950063325644,
 0.2728981618369436,
 0.48214641658656654,
 0.9614673714796995,
 0.9274509631442783,
 0.319448791592817,
 0.5784094980725899,
 0.6512289449827822,
 0.10404091635139145,
 0.1599860020697983,
 0.6039014384982189,
 0.34662716847196395,
 0.976977294675767,
 0.34441967166211795,
 0.2545669268786487,
 0.8411255023752522,
 0.00977388949406155,
 0.9802277762617235,
 0.4644116400828252,
 0.4874621425571981,
 0.1384663043916019,
 0.8492139795855199,
 0.3973732059547731,
 0.25602285730465535,
 0.42740316316649396,
 0.458477572381225,
 0.5220726320998363,
 0.23727302618199297,
 0.736546055933876,
 0.3891517514044428,
 0.5835598140701086,
 0.10200029269840416,
 0.6275022834642717,
 0.3904379874570679,
 0.14151408979585445,
 0.31331056825956893,
 0.3281337936600853,
 0.09256139622911097,
 

In [ ]:
# 다중 슬롯 머신의 ARM을 구현, 모든 ARM은 같은 형태를 취한다.
# ARM은 보상을 주는 확률을 각각 고유하게 가지고 있다
class SlotArm() :
  # 생성자
  def __init__(self, p) :
    self.p = p
    pass
  # 멤버변수
  # 멤버함수
  def reward(self) :
    if self.p > random.random() :
      return 1.0
    else :
      return 0.0
  #   pass
  # pass

# 확률 50%로 보상을 주는 ARM을 만들어라
SlotArm(0.5)
SlotArm(0.1)
SlotArm(0.3)